In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [ ]:
### Output like Stream (predict 1 price)

In [ ]:
model = load_model("./HSI_LSTM_Analysis.keras")
scaler = MinMaxScaler(feature_range=(0, 1))
look_back = 150

In [ ]:
df_hist = pd.read_csv("./data/_output.csv")
data = df_hist.iloc[-look_back:]
data_set = scaler.fit_transform(data[["Close"]]).reshape(1, -1)

In [ ]:
pred = []
df_curr = pd.read_csv("./data/_output.csv")
for x in range(len(df_curr)):
    row = df_curr.iloc[[x]]
    curr = scaler.transform(row[["Close"]])
    data_set = np.concatenate((data_set, curr), axis=1)
    data_set = np.delete(data_set, 0, axis=1)
    data_test = np.reshape(data_set, (data_set.shape[0], data_set.shape[1], 1))
    data_pred = scaler.inverse_transform(model.predict(data_test))
    pred.append(data_pred[:, 0])

In [ ]:
plt.plot(df_curr["Close"], label="actual")
plt.plot(pred, label="predict")
plt.legend()
plt.show()

In [ ]:
### Output pre Stream (predict 10 price)

In [ ]:
model = load_model("./HSI_LSTM_Analysis_Multiple_Output.keras")
scaler = MinMaxScaler(feature_range=(0, 1))
look_back = 150
n_steps = 10

In [ ]:
df = pd.concat([df_hist.tail(look_back), df_curr], ignore_index=True)
data = scaler.fit_transform(df[["Close"]]).reshape(1, -1)
data.shape

In [ ]:
pred = []
start = 0
end = look_back
while end <= data.shape[1]:
    print("start {}, end {}", start, end)
    data_set = data[:, start : end]
    data_test = np.reshape(data_set, (data_set.shape[0], data_set.shape[1], 1))
    data_pred = scaler.inverse_transform(model.predict(data_test))
    pred.extend(data_pred.flatten().tolist())
    start = start + n_steps
    end = end + n_steps

In [ ]:
plt.plot(df_curr["Close"], label="actual")
plt.plot(pred, label="predict")
plt.legend()
plt.show()